In [1]:
!pip install nlp

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nlp
import random
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

c:\Users\User\anaconda3\envs\applied_math\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [23]:
validation = pd.read_csv('./validation.csv')
train = pd.read_csv('./training.csv')
test = pd.read_csv('./test.csv')

In [27]:
train_text = train['text']
train_labels = train['label']

In [115]:
X = train['text']
y = train['label']

X = X.to_frame()
y = y.to_frame()
X = X.reindex()
y = y.reindex()

In [28]:
import plotly.express as px

px.bar(train_labels.value_counts(ascending=True),template='plotly_white')

In [110]:
y.head()

,label
0,0
1,0
2,3
3,2
4,3


,text
0,i didnt feel humiliated
1,i can go from feeling so hopeless to so damned...
2,im grabbing a minute to post i feel greedy wrong
3,i am ever feeling nostalgic about the fireplac...
4,i am feeling grouchy
...,...
15995,i just had a very brief time in the beanbag an...
15996,i am now turning and i feel pathetic that i am...
15997,i feel strong and good overall
15998,i feel like this was such a rude comment and i...


In [116]:
tokenizer = Tokenizer(num_words=10000, oov_token='<UNK>', split=' ', char_level=False)
tokenizer.fit_on_texts(X['text'])

In [117]:
tokenizer.texts_to_sequences(X.iloc[0])

[[2, 139, 3, 679]]

In [118]:
maxlen=50
def get_sequences(tokenizer, train_text):
    sequences = tokenizer.texts_to_sequences(train_text)
    padded = pad_sequences(sequences, truncating = 'post', padding='post', maxlen=maxlen)
    return padded

In [119]:
padded_train_seq = get_sequences(tokenizer, X['text'])

In [120]:
padded_train_seq.shape

(16000, 50)

In [121]:
y.shape

(16000, 1)

In [123]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter


print("Before undersampling:", Counter(y['label']))

class_counts = Counter(y['label'])

# Set the target number of samples per class
target_samples = 3000

# Determine classes needing oversampling and undersampling
classes_to_resample = {cls: target_samples for cls, count in class_counts.items() if count < target_samples}
classes_to_keep = {cls: target_samples for cls, count in class_counts.items() if count >= target_samples}
print(classes_to_resample)
print(classes_to_keep)
# Oversample minority classes
oversampler = RandomOverSampler(sampling_strategy=classes_to_resample)

# Undersample majority classes
undersampler = RandomUnderSampler(sampling_strategy=classes_to_keep)

# Apply both oversampling and undersampling
text_resampled, labels_resampled = oversampler.fit_resample(padded_train_seq, y)
labels_resampled = labels_resampled['label']
print(Counter(labels_resampled))


text_resampled, labels_resampled = undersampler.fit_resample(text_resampled, labels_resampled)

# Check the class distribution after resampling
print(Counter(labels_resampled))


Before undersampling: Counter({1: 5362, 0: 4666, 3: 2159, 4: 1937, 2: 1304, 5: 572})
{3: 3000, 2: 3000, 5: 3000, 4: 3000}
{0: 3000, 1: 3000}
Counter({1: 5362, 0: 4666, 3: 3000, 2: 3000, 5: 3000, 4: 3000})
Counter({0: 3000, 1: 3000, 2: 3000, 3: 3000, 4: 3000, 5: 3000})


In [ ]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter


print("Before undersampling:", Counter(y['label']))

class_counts = Counter(y['label'])

# Set the target number of samples per class
target_samples = 1000

# Determine classes needing oversampling and undersampling
classes_to_resample = {cls: target_samples for cls, count in class_counts.items() if count < target_samples}
classes_to_keep = {cls: target_samples for cls, count in class_counts.items() if count >= target_samples}
print(classes_to_resample)
print(classes_to_keep)
# Oversample minority classes
oversampler = RandomOverSampler(sampling_strategy=classes_to_resample)

# Undersample majority classes
undersampler = RandomUnderSampler(sampling_strategy=classes_to_keep)

# Apply both oversampling and undersampling
train_text_resampled, train_labels_resampled = oversampler.fit_resample(padded_train_seq, y)
train_labels_resampled = train_labels_resampled['label']
print(Counter(train_labels_resampled))


train_text_resampled, train_labels_resampled = undersampler.fit_resample(train_text_resampled, train_labels_resampled)

# Check the class distribution after resampling
print(Counter(train_labels_resampled))


In [138]:
labels_resampled = np.array(labels_resampled)

In [156]:
from sklearn.model_selection import train_test_split


X_train, X_temp, y_train, y_temp = train_test_split(text_resampled,labels_resampled,test_size=0.3,random_state=42)
X_val,X_test, y_val,y_test =  train_test_split(X_temp,y_temp,test_size=0.5, random_state=42)

[1 5 1 ... 1 0 5]


In [50]:
px.bar(train_labels_resampled.value_counts(ascending=True),template='plotly_white')
print(train_text_resampled[0], train_labels_resampled[0])

NameError: name 'px' is not defined

In [38]:
print(train_text_resampled)

[[  2 269   5 ...   0   0   0]
 [  2  24 136 ...   0   0   0]
 [  2 231   3 ...   0   0   0]
 ...
 [  2  72   3 ...   0   0   0]
 [  2  47  15 ...   0   0   0]
 [  2 323   6 ...   0   0   0]]


In [39]:
print(train_labels_resampled)

0       0
1       0
2       0
3       0
4       0
       ..
5995    5
5996    5
5997    5
5998    5
5999    5
Name: label, Length: 6000, dtype: int64


In [51]:
classes = ['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']
class_to_index = dict((c,i) for i, c in enumerate(classes))
index_to_class = dict((v,k) for k, v in class_to_index.items())
names_to_ids = lambda labels: np.array([class_to_index.get(x) for x in labels])

In [52]:
print(classes)

['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']


In [53]:
class_to_index

{'sadness': 0, 'joy': 1, 'love': 2, 'anger': 3, 'fear': 4, 'surprise': 5}

In [54]:
index_to_class

{0: 'sadness', 1: 'joy', 2: 'love', 3: 'anger', 4: 'fear', 5: 'surprise'}

In [44]:
print(train_labels_resampled[0])

0


In [92]:
val_text = validation['text']
val_labels = validation['label']
val_seq = get_sequences(tokenizer, val_text)
print(val_seq.shape)

(2000, 50)


In [94]:
class_counts = Counter(val_labels)
target_samples = 100

# Determine classes needing oversampling and undersampling
classes_to_resample = {cls: target_samples for cls, count in class_counts.items() if count < target_samples}
classes_to_keep = {cls: target_samples for cls, count in class_counts.items() if count >= target_samples}
print(classes_to_resample)
print(classes_to_keep)
# Oversample minority classes
oversampler = RandomOverSampler(sampling_strategy=classes_to_resample)

# Undersample majority classes
undersampler = RandomUnderSampler(sampling_strategy=classes_to_keep)


# Apply both oversampling and undersampling
val_text_resampled, val_labels_resampled = oversampler.fit_resample(val_seq, val_labels)
print(Counter(val_labels_resampled))


val_text_resampled, val_labels_resampled = undersampler.fit_resample(val_text_resampled, val_labels_resampled)

# Check the class distribution after resampling
print(Counter(val_labels_resampled))


{5: 100}
{0: 100, 2: 100, 3: 100, 1: 100, 4: 100}
Counter({1: 704, 0: 550, 3: 275, 4: 212, 2: 178, 5: 100})
Counter({0: 100, 1: 100, 2: 100, 3: 100, 4: 100, 5: 100})


In [77]:
train_text_resampled.shape

(6000, 50)

In [84]:
import plotly.express as px

px.bar(val_labels.value_counts(ascending=True),template='plotly_white')

In [158]:


model = tf.keras.models.Sequential([
tf.keras.layers.Embedding(10000,64,input_length=maxlen),
tf.keras.layers.SpatialDropout1D(0.2),
tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(80,dropout = 0.2, recurrent_dropout=0.2)),
tf.keras.layers.Dense(6, activation='sigmoid')
])
model.compile(
     loss='sparse_categorical_crossentropy',
     #optimizer='adam',
     optimizer=tf.keras.optimizers.Adam(learning_rate=5e-4),
     metrics=['accuracy']
)
model.summary()

h = model.fit(
     #train_text_resampled, train_labels_resampled,
     X_train, y_train,
     batch_size = 20,
     validation_data=(X_val, y_val),

     epochs=30,
     callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss',min_delta=0.001, patience=5)]
)

Model: "sequential_38"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_39 (Embedding)    (None, 50, 64)            640000    
                                                                 
 spatial_dropout1d_12 (Spat  (None, 50, 64)            0         
 ialDropout1D)                                                   
                                                                 
 bidirectional_72 (Bidirect  (None, 160)               92800     
 ional)                                                          
                                                                 
 dense_38 (Dense)            (None, 6)                 966       
                                                                 
Total params: 733766 (2.80 MB)
Trainable params: 733766 (2.80 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/30
630/63

In [159]:
test_text = test['text']
test_labels=test['label']
test_seq = get_sequences(tokenizer, test_text)

print(test_seq)

model.evaluate(X_test,y_test)

[[ 17   8 203 ...   0   0   0]
 [ 17   1  11 ...   0   0   0]
 [  2 145  80 ...   0   0   0]
 ...
 [  2   3   9 ...   0   0   0]
 [ 17   8  38 ...   0   0   0]
 [  2   3  36 ...   0   0   0]]
85/85 [==============================] - 2s 22ms/step - loss: 0.2488 - accuracy: 0.9322


[0.24878419935703278, 0.9322222471237183]

In [49]:
model.save("./emotion_model.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning:

You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.

